In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader

# 下载训练集
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=False)
# 下载测试集
test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=False)

# 设置批次数
batch_size = 100

# 装载训练集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
# 装载测试集
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)


# 自定义手写数字识别网络
class fnn(nn.Module):
    def __init__(self):
        super(fnn, self).__init__()

        self.layer1 = nn.Sequential(  # 全连接层     [1, 28, 28]
            nn.Linear(784, 400),  # 输入维度，输出维度
            nn.BatchNorm1d(400),  # 批标准化，加快收敛，可不需要
            nn.ReLU()  # 激活函数
        )

        self.layer2 = nn.Sequential(
            nn.Linear(400, 200),
            nn.BatchNorm1d(200),
            nn.ReLU()
        )

        self.layer3 = nn.Sequential(  # 全连接层
            nn.Linear(200, 100),
            nn.BatchNorm1d(100),
            nn.ReLU()
        )

        self.layer4 = nn.Sequential(  # 最后一层为实际输出，不需要激活函数，因为有 10 个数字，所以输出维度为 10，表示10 类
            nn.Linear(100, 10),
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        output = self.layer4(x)

        return output


# 定义学习率
LR = 0.1

# 定义一个网络对象
net = fnn()

# 损失函数使用交叉熵
loss_function = nn.CrossEntropyLoss()

# 优化函数使用 SGD
optimizer = optim.SGD(
    net.parameters(),
    lr=LR,
    momentum=0.9,
    weight_decay=0.0005
)

# 定义迭代次数
epoch = 20

# 进行迭代训练
for epoch in range(epoch):
    for i, data in enumerate(train_loader):
        inputs, labels = data

        # 转换下输入形状
        inputs = inputs.reshape(batch_size, 784)

        inputs, labels = Variable(inputs), Variable(labels)
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 初始化正确结果数为0
    test_result = 0

    # 用测试数据进行测试
    for data_test in test_loader:
        images, labels = data_test

        # 转换下输入形状
        images = images.reshape(batch_size, 784)

        images, labels = Variable(images), Variable(labels)
        output_test = net(images)

        # 对一个批次的数据的准确性进行判断
        for i in range(len(labels)):

            # 如果输出结果的最大值的索引与标签内正确数据相等，准确个数累加
            if torch.argmax(output_test[i]) == labels[i]:
                test_result += 1

    # 打印每次迭代后正确的结果数
    print("Epoch {} : {} / {}".format(epoch, test_result, len(test_dataset)))

# 保存权重模型
torch.save(net, './test.pkl')


D:\software\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Epoch 0 : 9649 / 10000
Epoch 1 : 9657 / 10000
Epoch 2 : 9727 / 10000
Epoch 3 : 9658 / 10000
Epoch 4 : 9732 / 10000
Epoch 5 : 9717 / 10000
Epoch 6 : 9747 / 10000
Epoch 7 : 9733 / 10000
Epoch 8 : 9731 / 10000
Epoch 9 : 9746 / 10000
Epoch 10 : 9728 / 10000
Epoch 11 : 9673 / 10000
Epoch 12 : 9747 / 10000
Epoch 13 : 9727 / 10000
Epoch 14 : 9737 / 10000
Epoch 15 : 9674 / 10000
Epoch 16 : 9686 / 10000
Epoch 17 : 9755 / 10000
Epoch 18 : 9725 / 10000
Epoch 19 : 9759 / 10000
